<h1><center>Mini-Lab: Logistic Regression and SVMs</center></h1>
<h3><center>A Deeper Analysis of Covid-19 Data</center></h3>
<p><center>DS 7331</center></p>
<p><center>Created by Sadik Aman, Dawn Bowerman, Zachary Harris, Alexandre Jasserme</center></p>

<p><center>Sections of this code was adapted from: 
    <li>https://nbviewer.org/github/jakemdrew/DataMiningNotebooks/blob/master/04.%20Logits%20and%20SVM.ipynb</li>


## Creating the Models

### Summary
We created a logistic regression model with an ordinal response variable, and because our data was ordered by date, we split the data using TimeSeriesSplit. Then, we trained the data and created a confusion matrix with three rows "high", "medium", and "low". We propose that the low accuracy at this point is due to the extraction of 'new_cases_smoothed', which we removed because the attribute dominated all of the other data attributes.  
We then compared the attributes with and without scaling, and the scaling definitely improved the accuracy.  
We plotted the weights, and this is where we determined that 'new_cases_smoothed' was saturating the attribute weights. This further confirmed our decision to remove it.
We experimented with the pipeline function in sklearn, created the pipeline, and graphed the logistical regression weights from the process. We also compared a few of the low-weight attributes to check for overfitting.  
After scaling, we ran the Support Vector Machine (SVM) function from sklearn.svm. Then, we trained the model again for the ordinal response variable using decision_function_shape = 'ovo' and labels "high", "medium", and "low".   It was slightly slower, but accuracy was better for the SVM at 0.62.
We made a dataframe of the training data and graphed the KDE of different Support Vector attributes compared to the original attributes.
We did not use Stochastic Gradient Descent as our dataset size was manageable.

### Code Execution

In [1]:
from __future__ import print_function

#Libraries
import plotly
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3
from matplotlib.pyplot import figure
from sklearn.model_selection import TimeSeriesSplit
# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
from sklearn.model_selection import ShuffleSplit

df = pd.read_csv('data/owid-covid-data_modified.csv') # read in the csv file

In [2]:
# Ideas from https://scikit-learn.org/stable/modules/cross_validation.html#timeseries-cv

# Sorting data frame by date column
df['date'] = pd.to_datetime(df['date']) # Converting data columnn to datetime

df = df.sort_values(by='date', ascending=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34965 entries, 0 to 34964
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   continent                       34965 non-null  object        
 1   location                        34965 non-null  object        
 2   date                            34965 non-null  datetime64[ns]
 3   new_cases                       34965 non-null  float64       
 4   new_cases_smoothed              34965 non-null  float64       
 5   new_deaths                      34965 non-null  float64       
 6   new_deaths_smoothed             34965 non-null  float64       
 7   reproduction_rate               34965 non-null  float64       
 8   new_vaccinations_smoothed       34965 non-null  float64       
 9   new_people_vaccinated_smoothed  34965 non-null  float64       
 10  stringency_index                34965 non-null  float64       
 11  po

#### Variable Elimination

Using the table above, we eliminated particular variables because they would not be appropriate for the model. The dropped variables are as follows:

*Unnamed: 0*: This variable was an index in the original dataset. This has no prediction value for the model.

*new_deaths*: This variable was eliminated due to high correlation with new_deaths_range, our response variable.

*new_deaths_smoothed*: This variable was eliminated for the same reason new_deaths was eliminated. 

*new_cases*: This variable was eliminated due to high correlation with new_deaths. 

*new_cases_smoothed*: This variable was eliminated due to high correlation with new_deaths. If we keep it in the model, it will overwhelm the other attributes - see the figure below.

*continent*: The variable type for this attribute is object.

*location*: The variable type for this attribute is object.

*stringency_range*:The variable type for this attribute is object.

*new_cases_range*: This variable was created as a potential reponse variable. The variable type for this attribute is object.  

*date*: While this variable was used to sort the dataset initially, it has to be eliminated when incorporating the dataset into the model due to the variable type.

*new_people_vaccinated_smoothed*: This variable was eliminated due to high correlation with new_vaccinations_smoothed.

*population_density*: This variable was eliminated due to high correlation with population.

*aged_70_older*: This variable was eliminated due to high correlation with aged_65_older. 

*cardiovasc_death_rate*: This variable had a low logistic regression weight.







<table style ="width: 100%"><tr>
    <td> <img src="images/weights.png" style="width:100%"> </td>
    <td> <img src="images/acc_new_cases_smoothed.png" style="width:60%"> </td>
</tr></table>

The figure above shows that new_cases_smoothed will dominate the other attributes if we keep it in the model.
After examining the logistic regression weights, we realized that new_cases_smoothed has a heavy influence on the model. This could create bias among the predictors. While including new_cases_smoothed improved accuracy, we dropped it from the dataset due to its dominating effects.

In [3]:
float_df = df.drop(["new_deaths", 
                    "new_deaths_smoothed",
                    "new_cases",
                    "new_cases_smoothed",
                    "continent",
                    "location",
                    "stringency_range",
                    "new_cases_range",
                    "date",
                    "new_people_vaccinated_smoothed",
                    "population_density",
                    "aged_70_older",
                    "cardiovasc_death_rate"], axis=1)

float_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34965 entries, 0 to 34964
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   reproduction_rate           34965 non-null  float64
 1   new_vaccinations_smoothed   34965 non-null  float64
 2   stringency_index            34965 non-null  float64
 3   population                  34965 non-null  int64  
 4   median_age                  34965 non-null  float64
 5   aged_65_older               34965 non-null  float64
 6   gdp_per_capita              34965 non-null  float64
 7   diabetes_prevalence         34965 non-null  float64
 8   handwashing_facilities      34965 non-null  float64
 9   hospital_beds_per_thousand  34965 non-null  float64
 10  life_expectancy             34965 non-null  float64
 11  human_development_index     34965 non-null  float64
 12  new_deaths_range            34965 non-null  object 
dtypes: float64(11), int64(1), objec

#### Predicting and Classifying

In [4]:
# to predict the X and y data as follows:
if 'new_deaths_range' in float_df:
    y = float_df['new_deaths_range'].values # get the labels
    del float_df['new_deaths_range'] # get rid of the class label
    X = float_df.values # use everything else to predict
       ## X and y are now numpy matrices, by calling 'values' on the pandas data frames they 
    #    have converted into simple matrices to use with scikit learn

In [5]:
# create a reusable logisitic regression object
#   setup the object with different learning parameters and constants
# Our classes are imbalanced. To address this situation, we are setting the class_weight parameter to 'balanced'
# We are setting solver to 'lbfgs' and multi_class to 'multinomial' because our response variable is ordinal instead of binomial
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight="balanced", max_iter=500, solver='lbfgs', multi_class='multinomial' ) # get object

In [6]:
iter_num=0
# the indices are the rows used for training and testing in each iteration

cv_object = TimeSeriesSplit( n_splits=3)

# TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
for train_index, test_index in cv_object.split(X):
   
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index] 

    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set predictions
    
# accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat,labels=["high","medium","low"] )
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    iter_num+=1

====Iteration 0  ====
accuracy 0.3556801281317927
confusion matrix
 [[1409  783  910]
 [ 660 1246  578]
 [ 929 1772  454]]
====Iteration 1  ====
accuracy 0.2963047706212104
confusion matrix
 [[1079 1136  753]
 [ 750 1239  257]
 [1155 2100  272]]
====Iteration 2  ====
accuracy 0.26049651069671664
confusion matrix
 [[ 768  967  930]
 [ 843 1125  256]
 [1276 2192  384]]


After dropping the new_cases_smoothed variable, we noticed that the model had increased difficulty classifying the medium death_range. We believe this is due to the model losing its strongest predictor, new_cases_smoothed. Also, we addressed the unbalanced classes by setting the class_weight parameter to 'balanced'.

#### Calculating the Weights

In [7]:
# iterate over the coefficients
weights = lr_clf.coef_.T # take transpose to make a column vector
variable_names = float_df.columns
for coef, name in zip(weights,variable_names):
    print(name, 'has weight of', coef[0])

reproduction_rate has weight of -1.6423614023459299e-10
new_vaccinations_smoothed has weight of 6.583034995076703e-07
stringency_index has weight of 1.142487864663763e-09
population has weight of 2.1796281351136897e-09
median_age has weight of -1.5001143082542263e-09
aged_65_older has weight of 7.534805202699801e-10
gdp_per_capita has weight of -5.277888086444772e-06
diabetes_prevalence has weight of -1.4029643458457881e-09
handwashing_facilities has weight of 2.6276149607966282e-09
hospital_beds_per_thousand has weight of -8.356025662838866e-11
life_expectancy has weight of -1.0050323621522008e-08
human_development_index has weight of -6.784987115641003e-11


When first calculating the weights, the attribute weights were not normalized. In the following section, we scaled the attribute weights to improve predictive power.

#### Scaling the Logistic Regression Weights

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
# scale attributes by the training set
scl_obj = StandardScaler()
scl_obj.fit(X_train) # find scalings for each column that make this zero mean and unit std

X_train_scaled = scl_obj.transform(X_train) # apply to training

In [10]:
X_test_scaled = scl_obj.transform(X_test) # apply those means and std to the test set (without snooping at the test set values)

# train the model just as before
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight="balanced", max_iter=500, solver='lbfgs', multi_class='multinomial' )
lr_clf.fit(X_train_scaled,y_train)  # train object

y_hat = lr_clf.predict(X_test_scaled) # get test set precitions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat, labels = ["high","medium","low"])
print('accuracy:', acc )
print(conf )

accuracy: 0.523281089120238
[[1804  413  448]
 [ 564  743  917]
 [ 429 1396 2027]]


Compared to the previous two models, scaling the X_train data seems to improve the overall model. Results from this model show improved accuracy.

In [11]:
zip_vars = zip(lr_clf.coef_.T,float_df.columns) # combine attributes
zip_vars = sorted(zip_vars, key=lambda x: x[1]) # sorting them alphabetically

print ('The logistic regression weights for the scaled attributes are the following')
for coef, name in zip_vars:
    print(name, 'has weight of', coef[0])

The logistic regression weights for the scaled attributes are the following
aged_65_older has weight of 0.1947991599209367
diabetes_prevalence has weight of -0.11410240835743592
gdp_per_capita has weight of -0.6886735701314174
handwashing_facilities has weight of 0.1134215085553507
hospital_beds_per_thousand has weight of -0.09584048580292809
human_development_index has weight of 0.5478867941684469
life_expectancy has weight of -0.14329359110245216
median_age has weight of 0.2483165919030426
new_vaccinations_smoothed has weight of 0.3105734829994389
population has weight of 0.565364461335144
reproduction_rate has weight of -0.11332845179716328
stringency_index has weight of 0.37957597467397325


In [12]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

weights = pd.Series(lr_clf.coef_[0],index=float_df.columns)
weights.plot(kind='bar')
plt.show()

## Interpreting Feature Importance

Discussions of the weights:
    
*A positive weight indicates a higher probability that the number  of new deaths will have a high range.* 

* aged_65_older has weight of 0.1947991599212599
* median_age has weight of 0.24831659190249658  
The weights for these 2 features make sense: a country with an older population will have a higher probability of having more people dying from the Covid-19.  


* human_development_index has weight of 0.5478867941689486
* population has weight of 0.5653644613353508  
These 2 features have the strongest  positive weight. Countries with high population are more likely to have big cities with a high population density. Also, the human development index includes the years of schooling and schools can be a significant vector of transmission for a respiratory disease like the Covid-19.  


* handwashing_facilities has weight of 0.11342150855533747
* new_vaccinations_smoothed has weight of 0.3105734829999393
* stringency_index has weight of 0.3795759746738224  
The weights for these 3 features are counterintuitive: one would expect that they would decrease the probability of a high number of deaths instead of increasing it. The handwashing_facilities feature has a low weight and since Covid-19 is mostly transmitted by airborne particles and droplets so hand washing would not be very relevant to prevent its transmission. The other 2 features, new_vaccinations_smoothed and stringency_index, are probably related to public policy reaction. Since we are analyzing data that was collected over time, it is probable that a country that has experienced a high number of deaths related to Covid-19 will be more likely to implement stronger restrictions on the population and encourage or mandate vaccinations.


*A negative weight indicates a higher probability that the number  of new deaths will have a low range.*
* gdp_per_capita has weight of -0.6886735701313244
* hospital_beds_per_thousand has weight of -0.09584048580284012
* life_expectancy has weight of -0.1432935911024816  
These 3 features, which include the highest negative weight (represented by the lowest negative number) for gdp_per_capita, makes sense: wealthy countries, countries with a high number of hospital beds and countries with high life expectancy have a greater probability of having fewer deaths from Covid-19.


* diabetes_prevalence has weight of -0.11410240835732838
* reproduction_rate has weight of -0.1133284517970738  
We are not sure why a high prevalence of diabetes would increase the probability of having a low number of deaths. Also, the negative weight for reproduction_rate could be related to the Omicron variant (more transmissible compared to the previous variants but causing significantly fewer deaths). Note that the weights associated with these 2 features is pretty low.


Another factor to keep in mind is that we are trying to predict the range for the number of deaths from Covid-19 and this may not be precisely recorded in every country. There are likely countries where deaths are not categorized correctly, either because someone died with Covid-19 but it was not the main cause of death, or the opposite scenario where someone dies from Covid-19 but it was not diagnosed as the cause of death.  


Note that this analysis is only valid because we removed new_cases_smoothed from the data set used to build the model. Since the number of new cases has a high positive correlation with the number of deaths (see the scatter plot below), the weight of this feature would overwhelm all the other features.

<img src="images/new_cases_death.png" style="width:65%"> 

<imng src="images/weights.png">

### Experimenting with Pipeline

In [13]:
from sklearn.pipeline import Pipeline
# you can apply the StandardScaler function inside of the cross-validation loop 
#  but this requires the use of PipeLines in scikit. 
#  A pipeline can apply feature pre-processing and data fitting in one compact notation
#  Here is an example!

std_scl = StandardScaler()
lr_clf = LogisticRegression(penalty='l2', C=1.0, solver='lbfgs') 

# create the pipline
piped_object = Pipeline([('scale', std_scl),  # do this
                         ('logit_model', lr_clf)]) # and then do this

weights = []
# run the pipline cross validated
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
    piped_object.fit(X[train_indices],y[train_indices])  # train object
    # it is a little odd getting trained objects from a  pipeline:
    weights.append(piped_object.named_steps['logit_model'].coef_[0])
    

weights = np.array(weights)
import plotly
plotly.offline.init_notebook_mode() # run at the start of every notebook

error_y=dict(
            type='data',
            array=np.std(weights,axis=0),
            visible=True
        )

graph1 = {'x': float_df.columns,
          'y': np.mean(weights,axis=0),
    'error_y':error_y,
       'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Logistic Regression Weights, with error bars'}

plotly.offline.iplot(fig)


Using the pipeline method, we were able to produce another graphical representation of the logistic regression weights. This graph includes error bars. We created another graph using the pipeline to help us improve and guard against overfitting.

In [14]:
Xnew = float_df[['handwashing_facilities','hospital_beds_per_thousand']].values

weights = []
# run the pipline crossvalidated
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(Xnew,y)):
    piped_object.fit(Xnew[train_indices],y[train_indices])  # train object
    weights.append(piped_object.named_steps['logit_model'].coef_[0])
    
weights = np.array(weights)

error_y=dict(
            type='data',
            array=np.std(weights,axis=0),
            visible=True
        )

graph1 = {'x': ['handwashing_facilities','hospital_beds_per_thousand'],
          'y': np.mean(weights,axis=0),
    'error_y':error_y,
       'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Logistic Regression Weights, with error bars'}

plotly.offline.iplot(fig)

Reviewing the logistic regression weights, we noticed that handwashing_facilities and hospital_beds_per_thousand had neglible weight values. 

## Support Vector Machines

#### Creating the Model Using Support Vector Machines

In [ ]:
# cross validation loop and set the training and testing variable for one single iteration
for train_indices, test_indices in cv_object.split(X,y): 
    
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
X_train_scaled = scl_obj.transform(X_train) # apply to training
X_test_scaled = scl_obj.transform(X_test) 
# SVMs on the data and play with the parameters and kernels
from sklearn.svm import SVC

# train the model
svm_clf = SVC(C=0.5, kernel='rbf', decision_function_shape='ovo', degree=3, gamma='auto') # get object
svm_clf.fit(X_train_scaled, y_train)  # train object

y_hat = svm_clf.predict(X_test_scaled) # get test set predictions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat, labels = ["high","medium","low"])
print('accuracy:', acc )
print(conf)

#### Examining the Support Vectors

In [ ]:
print(svm_clf.support_vectors_.shape)
print(svm_clf.support_.shape)
print(svm_clf.n_support_ )

In [ ]:
# make a dataframe of the training data
df_tested_on = float_df.iloc[train_indices].copy() # saved from above, the indices chosen for training
# get the support vectors from the trained model
df_support = df_tested_on.iloc[svm_clf.support_,:].copy()

df_support['new_deaths_range'] = y[svm_clf.support_] # add back in the 'new_deaths_range' Column to the pandas dataframe
float_df['new_deaths_range'] = y # also add it back in for the original data
df_support.info()

In [ ]:
# statistics of the attributes
from pandas.plotting import boxplot

# group the original data and the support vectors
df_grouped_support = df_support.groupby(['new_deaths_range'])
df_grouped = float_df.groupby(['new_deaths_range'])

# plot KDE of Different variables
vars_to_plot = ['stringency_index','population','gdp_per_capita','human_development_index']

for v in vars_to_plot:
    plt.figure(figsize=(20,4))
    # plot support vector stats
    plt.subplot(1,2,1)
    ax = df_grouped_support[v].plot.kde() 
    plt.legend(['low','medium','high'])
    plt.title(v+' (Instances chosen as Support Vectors)')
    
    # plot original distributions
    plt.subplot(1,2,2)
    ax = df_grouped[v].plot.kde() 
    plt.legend(['low','medium','high'])
    plt.title(v+' (Original)')

## Intepreting Support Vectors

We have selected four instances as support vectors, and we have assessed the geometrical properties of each instance to show the insights. We implemented the RBF kernel to transform the data, OVO strategy between target variable(new_deaths_range), and selected SVM instances for this process. We also used KDE to estimate the probability density function of the selected instances.

As we can see from the density plot, there is no significant difference between the highest/peak points in both SVM and LR plots. Instead, the peak point ranges are similar. However, we can see that density plots for the SVM instances of reproduction_rate, stringency_index, and human_developmnet_indexs are more sparsed than the original plots. This is because SVMs try to maximize the margin between the closest support vectors whereas logistic regression models try to maximize the posterior class probability.

We can also see that the accuracy of our SVM is higher than the accuracy of our LR. This is because SVM tries to find the best margin that helps reduce the risk of error, whereas LR may have different decision boundaries with different weights near to maximum (optimal) point.

We did not use Stochastic Gradient Descent for this anaylsis. If the dataset used for this analysis had been massive, using SGD may have been appropriate. Since the dataset was maneagble in size, this was not needed.

------

## Model Advantages

Comparing the support vector machine (SVM) model for the classification task to the logistic regression model, we find that logistic regression was a little faster. However, both models' training times were quick (less than a minute). This was no doubt because logistic regression is more efficient in training the model than SVM. However, the prediction accuracy for SVM was better than logistic regression for the dataset;  after scaling, it was 0.52 for logistic regression and 0.62 for SVM.    
A discussion of the advantages and disadvantages of Logistic Regression and SVM is below:

**Logistic Regression:**
* Logistic regression is easier to perform, interpret, and very efficient to train when the dataset has linearly separable attributes.
* It is very fast at classifying unknown records.
* It can measure predictor coefficient size and direction. 
* It is used on an input variable where the output is a discrete value between 1 and 0.
* It can easily extend to multiple classes (multinomial regression) 
* Can be used for:  Pass or fail, Cancer detection, marketing.
* Logistic regression does not use a hyperplane decision boundary. Instead, it can have different decision boundaries with different weights near the optimal point.
* Logistic regression does not work well with unstructured and semi-structured data like text and images. It works with already identified independent variables.
* Logistic regression is based on the statistical properties of the data.
* Logistic regression can be vulnerable to overfitting.


**Support Vector Machine (SVM):**
* Used for both classification and regression problems, but mainly classification.
* Uses a hyperplane decision boundary that separates data into classes.  
* Uses the kernel trick to find the best line separator (clear and more powerful).
* Can be used for: Image classification, recognizing handwriting, and cancer detection.
* SVM tries to find the best margin that separates the classes, reducing the risk of error. SVM, therefore, has better accuracy than most other algorithms.
* SVM is not as prone to outliers as it only cares about the points closest to the decision boundary.
* SVM works well with unstructured and semi-structured data like text and images.
* SVM is based on the geometrical properties of the data.
* The risk of overfitting is less with SVM.

These are dataset guidelines for when to use Logistic regression or SVM:
If you have many attributes and your observations are 10 to10,000 (small)  use logistic regression or SVM with a linear kernel.

If your attributes are few (1 - 1000) and your observations are intermediate (10 - 10000), use SVM with Gaussian, polynomial, etc.) kernel.

If your attributes are few (1 - 1000) and your observations are large (50000 - 1000000, try adding more attributes and then use logistic regression or SVM with a linear kernel.

Note: It is standard to first try logistic regression. Then if it fails, try SVM without a kernel (know as SVM with a linear kernel).  Both of these methods are similar.